Until we properly handle the **categorical variables** with label encoder or one hot encoder we will be using just the continuous vars

#### Data description
- **TransactionDT**: timedelta from a given reference datetime (not an actual timestamp)
- **TransactionAMT**: transaction payment amount in USD
- **ProductCD**: product code, the product for each transaction
- **card1 - card6**: payment card information, such as card type, card category, issue bank, country, etc.
- **addr**: address
- **dist**: distance
- **P_ and (R__) emaildomain**: purchaser and recipient email domain
- **C1-C14**: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- **D1-D15**: timedelta, such as days between previous transaction, etc.
- **M1-M9**: match, such as names on card and address, etc.
- **Vxxx**: Vesta engineered rich features, including ranking, counting, and other entity relations.

##### Categorical Features:
ProductCD  
card1 - card6  
addr1, addr2  
Pemaildomain Remaildomain  
M1 - M9  

In [21]:
import sys
sys.path.append('..')
from utils import plot_counts_and_proportion, read_data

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline 

In [7]:
# Metrics
from sklearn.metrics import roc_auc_score, f1_score, log_loss
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


Precision = (TP)/(TP+FP)

Recall = (TP)/(TP+FN) 


We want to capture as many positives as possible, so recall could be an option, the problem with recall 
is that Recall is 1 if we predict 1 for all examples.

f1_score = $2\frac{precision*recall}{precision + recall}$

weighted f1 score  
  
f1_score = $(1 + \beta^2)\frac{precision*recall}{\beta^2 precision + recall}$

represents the importance that me give to recall over precision

Let try $\beta$ > 9.7

## Data read


In [9]:
folder_path = '~/Documents/Proyects/aprendizaje-estadistico/ieee-fraud-detection/input/'

# Train
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv', nrows=30000)
X, y = train_transaction.drop(['isFraud', 'TransactionID'], axis=1), train_transaction['isFraud']

In [44]:
categorical_vars = set(['ProductCD', *[f'card{i}' for i in range(1,7)],
                   *[f'M{i}' for i in range(1,10)], 
                     'P_emaildomain', 'R_emaildomain', 'addr1', 'addr2'])

continuous_vars = X.columns.difference(categorical_vars)

X = X[continuous_vars]
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values.ravel(), test_size=0.2, random_state=42)


In [22]:
X_tr, X2, y1, y2 = read_data()

NameError: name 'pd' is not defined

In [50]:
print(f'''
X_train shape:  {X_train.shape},
y_train shape: {y_train.shape}

X_test shape: {X_test.shape},
y_test shape: {y_test.shape}
''')


X_train shape:  (24000, 372),
y_train shape: (24000,)

X_test shape: (6000, 372),
y_test shape: (6000,)



In [59]:
def train_model(model, X, y):
    return model().fit(X, y)

In [62]:
# train_model(LogisticRegression, X_train, y_train)
train_model(GaussianNB, X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [38]:
y_train.ravel()

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
y_train.head()

21753    0
251      0
22941    0
618      0
17090    0
Name: isFraud, dtype: int64